In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

from scipy.stats import binom
from sklearn.linear_model import LinearRegression

pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 40)

In [75]:
# Collecte des données

data=pd.read_csv("Donnees-sur-le-parc-de-vehicule-au-niveau-regional.2023-05.csv", sep=';',header=[1])
zones = gpd.read_file('aires.geojson')

In [76]:
variables = ['PARC_2011','PARC_2012','PARC_2013','PARC_2014','PARC_2015','PARC_2016','PARC_2017','PARC_2018','PARC_2019','PARC_2020','PARC_2021','PARC_2022']
regions = data['REGION_LIBELLE'].unique()
regionsr = ['Auvergne-Rhône-Alpes', 'Bourgogne-Franche-Comté', 'Bretagne',
       'Centre-Val de Loire', 'Corse', 'Grand Est', 'Guadeloupe',
       'Guyane', 'Hauts-de-France', 'La Réunion',
       'Martinique', 'Mayotte', 'Normandie', 'Nouvelle-Aquitaine',
       'Occitanie', 'Pays de la Loire', "Provence-Alpes-Côte d'Azur",
       'Île-de-France']
regionsmet = ['Auvergne-Rhône-Alpes', 'Bourgogne-Franche-Comté', 'Bretagne',
       'Centre-Val de Loire', 'Grand Est', 
        'Hauts-de-France', 'Normandie', 'Nouvelle-Aquitaine',
       'Occitanie', 'Pays de la Loire', "Provence-Alpes-Côte d'Azur",
       'Île-de-France']
regionsmethorsb = ['Auvergne-Rhône-Alpes', 'Bretagne',
       'Centre-Val de Loire', 'Grand Est', 
        'Hauts-de-France', 'Normandie', 'Nouvelle-Aquitaine',
       'Occitanie', 'Pays de la Loire', "Provence-Alpes-Côte d'Azur",
       'Île-de-France']
critair = data['CRITAIR'].unique()

In [77]:
data.sample(5)

,REGION_CODE,REGION_LIBELLE,CLASSE_VEHICULE,CATEGORIE_VEHICULE,CARBURANT,AGE,STATUT_UTILISATEUR,CRITAIR,PARC_2011,PARC_2012,PARC_2013,PARC_2014,PARC_2015,PARC_2016,PARC_2017,PARC_2018,PARC_2019,PARC_2020,PARC_2021,PARC_2022
45363,76,Occitanie,vp,Véhicule particulier,Gaz et inconnu,3 ans,Particulier,Non classé,43,10,5,1,1,0,0,0,0,0,0,0
56339,75,Nouvelle-Aquitaine,pl,Camion,Gaz et inconnu,16 ans,Particulier,Non classé,0,0,0,0,0,0,0,0,0,0,0,0
45636,52,Pays de la Loire,vp,Véhicule particulier,Diesel,9 ans,Professionnel,Crit'Air 2,0,0,0,0,0,0,0,4,99,402,1636,1774
53310,32,Hauts-de-France,pl,Camion,Diesel,19 ans,Professionnel,Crit'Air 5,0,0,0,0,0,0,0,0,0,0,329,363
44443,75,Nouvelle-Aquitaine,vp,Véhicule particulier,Gaz et inconnu,3 ans,Particulier,Crit'Air 1,378,312,2617,7163,1218,157,284,217,173,28,83,238


In [ ]:
data_fil = data.loc[(data["CATEGORIE_VEHICULE"] == "Véhicule particulier") | (data["CATEGORIE_VEHICULE"] == "Dérivé VP") | (data["CATEGORIE_VEHICULE"] == "Camionnette")]
pannes = np.ndarray((len(data_fil),))
data_fil.groupby(['REGION_CODE','CATEGORIE_VEHICULE','CARBURANT','AGE'])

for i in range (len(data_fil)) :
    pannes_annees = []
    for j in range (11, 22) :
        var = "PARC_20" + str(j)
        var_2 = "PARC_20" + str(j + 1)
        age = int(data_fil.iloc[i]["AGE"][0:2])
        if age == 0 :
            age_plus_1 = "1 an"
        elif age == 24 :
            age_plus_1 = "25 ans et plus"
        else : 
            age_plus_1 = str(age + 1) + " ans"
        data_plus_1 = data_fil.loc[
                        (data_fil["REGION_CODE"] == data_fil.iloc[i]["REGION_CODE"]) &
                        (data_fil["CATEGORIE_VEHICULE"] == data_fil.iloc[i]["CATEGORIE_VEHICULE"]) &
                        (data_fil["CARBURANT"] == data_fil.iloc[i]["CARBURANT"]) &
                        (data_fil["CRITAIR"] == data_fil.iloc[i]["CRITAIR"]) &
                        (data_fil["STATUT_UTILISATEUR"] == data_fil.iloc[i]["STATUT_UTILISATEUR"]) &
                        (data_fil["AGE"] == age_plus_1)
        ]
        if len(data_plus_1) > 0 :
            pannes_annees.append(data_fil.iloc[i][var] - data_plus_1.iloc[0][var_2])
    pannes[i] = np.mean(pannes_annees)

data_fil.insert(8, 'Pannes', pannes)


In [ ]:
data_fil.head(5)